Code to reproduce running example of [LJP17]. It is based on a state-feedback control that
might undergo dropouts in its state feedback.

B. Legat, R. M. Jungers, and P. A. Parrilo
[**Certifying unstability of Switched Systems using Sum of Squares Programming**](https://arxiv.org/abs/1710.01814)

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %

In [3]:
using HybridSystems
include(joinpath(dirname(dirname(pathof(HybridSystems))), "examples", "PEDJ16s4.jl"))
using SwitchOnSafety

┌ Info: Recompiling stale cache file /home/blegat/.julia/compiled/v1.1/SwitchOnSafety/EUPLd.ji for SwitchOnSafety [ceb7f16a-07bf-5f4a-9354-b68f01b1610f]
└ @ Base loading.jl:1184


As showed in [Example 7, LJP16], the JSR is 0.9748171979372074 which is attained for the cycle
$$(3, 1, 2), (1, 3, 1), (3, 1, 2), (1, 2, 3), (2, 3, 1), (3, 3, 1)^3$$
which corresponds to the sequence of symbols: 21231111.

[LJP16] B. Legat, R. M. Jungers and P. A. Parrilo,
*Generating unstable trajectories for Switched Systems via Dual Sum-Of-Squares techniques.*
Proceedings of the 19th International Conference on Hybrid Systems: Computation and Control,
ACM, **2016**, 51-60.

In [4]:
function t(pair::Pair)
    edge = HybridSystems.edge_object(hs.automaton, pair.first, pair.second)
    # As there is no multiple edges, i.e. edges with same source and target but different label,
    # we can get the transition as follows
    @assert isone(length(hs.automaton.Σ[edge]))
    id = first(keys(hs.automaton.Σ[edge]))
    return HybridSystems.LightTransition(edge, id)
end
smp = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 1, 1 => 2, 2 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1])

The matrix 1 can be repeated since it is part of the self-loop (3, 3, 1) so it also provides a lower bound but it is not an s.m.p.

In [5]:
psw1 = periodicswitching(hs, t.([3 => 3]))

PSW(0.9392550239418471, [1])

The following periodic switchings have a high growth rates but are not s.m.p.

In [6]:
psw2 = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 2, 2 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.9659284377734763, [2, 1, 3, 1, 1, 1, 1])

In [7]:
psw3 = periodicswitching(hs, t.([3 => 1, 1 => 3, 3 => 1, 1 => 3, 3 => 3, 3 => 3, 3 => 3, 3 => 3]))

PSW(0.9728940109399586, [2, 1, 2, 1, 1, 1, 1, 1])

## Gripenberg

With a maximum length of 6, the Gripenberg algorithm finds `psw1`.

In [8]:
@time psw, ub = gripenberg(hs, max_length = 6)

ρ evaluations   : 425 < 1000 = max_ρ_eval.
norm evaluations: 1567 < 10000 = max_norm_eval.
switch length   : 6 ≥ 6 = max_length.
  1.785923 seconds (5.46 M allocations: 280.993 MiB, 5.63% gc time)


(PSW(0.9392550239418472, [1, 1, 1, 1, 1, 1]), 1.039593578184173)

With a maximum length of 7, the Gripenberg algorithm finds `psw2`.

In [9]:
@time psw, ub = gripenberg(hs, max_length = 7)

ρ evaluations   : 834 < 1000 = max_ρ_eval.
norm evaluations: 3137 < 10000 = max_norm_eval.
switch length   : 7 ≥ 7 = max_length.
  0.018071 seconds (160.82 k allocations: 10.914 MiB)


(PSW(0.9659284377734763, [1, 1, 1, 2, 1, 3, 1]), 1.0181518243613532)

With a maximum length of 8, the Gripenberg algorithm finds `psw3` but the upper bound is not so good.

In [10]:
@time psw, ub = gripenberg(hs, max_length = 8)

ρ evaluations   : 1134 ≥ 1000 = max_ρ_eval.
norm evaluations: 4296 < 10000 = max_norm_eval.
switch length   : 8 ≥ 8 = max_length.
  0.026258 seconds (219.43 k allocations: 14.940 MiB)


(PSW(0.9748171979372074, [1, 1, 1, 2, 1, 2, 3, 1]), 1.0011588099570934)

With a higher maximum length, the Gripenberg algorithm finds `psw3` and the upper bound is now exactly the lower bound + `1e-2`, the tolerance $\delta$.

In [11]:
@time psw, ub = gripenberg(hs, max_ρ_eval = 2000)

ρ evaluations   : 1196 < 2000 = max_ρ_eval.
norm evaluations: 4544 < 10000 = max_norm_eval.
switch length   : 9 < 50 = max_length.
  0.044325 seconds (245.52 k allocations: 16.567 MiB, 13.15% gc time)


(PSW(0.9748171979372074, [1, 1, 1, 2, 1, 2, 3, 1]), 0.9848171979372075)

We can reach an accuracy of `1e-4` without much troubles.

In [13]:
@time psw, ub = gripenberg(hs, max_ρ_eval = 2000, δ = 1e-4)

ρ evaluations   : 1571 < 2000 = max_ρ_eval.
norm evaluations: 6001 < 10000 = max_norm_eval.
switch length   : 10 < 50 = max_length.
  0.043460 seconds (306.12 k allocations: 20.914 MiB, 12.35% gc time)


(PSW(0.9748171979372074, [1, 1, 1, 2, 1, 2, 3, 1]), 0.9749171979372074)

But for `1e-5`, we need to go up to switching length 58.

In [19]:
@time psw, ub = gripenberg(hs, δ = 1e-5, max_ρ_eval = 10000, max_norm_eval = 20000, max_length = 60)

ρ evaluations   : 4938 < 10000 = max_ρ_eval.
norm evaluations: 18365 < 20000 = max_norm_eval.
switch length   : 58 < 60 = max_length.
  0.135948 seconds (960.17 k allocations: 73.438 MiB, 12.74% gc time)


(PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1]), 0.9748271979372074)

In [36]:
@time psw, ub = gripenberg(hs, δ = 1e-6, max_ρ_eval = 40000, max_norm_eval = 200000, max_length = 600)

ρ evaluations   : 37476 < 40000 = max_ρ_eval.
norm evaluations: 137845 < 200000 = max_norm_eval.
switch length   : 522 < 600 = max_length.
  0.960303 seconds (7.04 M allocations: 1.242 GiB, 14.94% gc time)


(PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1,

In [53]:
@time psw, ub = gripenberg(hs, δ = 0.25e-6, max_ρ_eval = 200000, max_norm_eval = 1000000, max_length = 3000)

ρ evaluations   : 145188 < 200000 = max_ρ_eval.
norm evaluations: 533365 < 1000000 = max_norm_eval.
switch length   : 2058 < 3000 = max_length.
  6.408790 seconds (30.31 M allocations: 13.984 GiB, 25.33% gc time)


(PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1,

## Polytopic proof of the s.m.p.

In [13]:
using JuMP, MosekTools
@time smp, done, polys = invariant_polytopes(hs, with_optimizer(Mosek.Optimizer, QUIET=true), smp,
                                             tol=1e-2, verbose=2, max_length=40);

Depth 1: [2, 1, 5, 0] points, 8 living leaves...
Depth 2: [6, 7, 7, 5] points, 17 living leaves...
Depth 3: [7, 9, 9, 7] points, 7 living leaves...
Depth 4: [8, 11, 10, 9] points, 6 living leaves...
Depth 5: [9, 12, 11, 10] points, 4 living leaves...
Depth 6: [10, 13, 12, 11] points, 4 living leaves...
Depth 7: [11, 14, 13, 12] points, 4 living leaves...
Depth 8: [12, 15, 14, 13] points, 4 living leaves...
Depth 9: [13, 16, 15, 14] points, 4 living leaves...
Depth 10: [14, 17, 16, 15] points, 4 living leaves...
Depth 11: [15, 18, 17, 16] points, 4 living leaves...
Depth 12: [16, 19, 18, 17] points, 4 living leaves...
Depth 13: [17, 20, 19, 18] points, 4 living leaves...
Depth 14: [18, 21, 20, 19] points, 4 living leaves...
Depth 15: [19, 21, 20, 20] points, 2 living leaves...
0 living leaves, PSW(0.9748171979372073, [2, 1, 2, 3, 1, 1, 1, 1]) is a spectral maximizing product (s.m.p.)
Use `tol` < 0.00884085255057343 to eliminate at least one more living leaf.
Use `tol` > 0.01163761611231

In [14]:
using JuMP, MosekTools
@time smp, done, polys = invariant_polytopes(hs, with_optimizer(Mosek.Optimizer, QUIET=true), psw1, tol=1e-6);

Depth 1: [0, 0, 1, 0] points, 1 living leaves...
Depth 2: [1, 1, 1, 1] points, 3 living leaves...
New candidate s.m.p. found because ⟨v_1*, v⟩ - 1 = 1.1889629260138844e16 - 6.124044746116224e15im > 1.0e-6:
PSW(0.9635728013202637, [1, 1, 1, 1, 1, 2, 1])
Depth 1: [1, 0, 6, 0] points, 7 living leaves...
Depth 2: [6, 7, 6, 6] points, 18 living leaves...
New candidate s.m.p. found because ⟨v_1*, v⟩ - 1 = 0.17314159011335306 > 1.0e-6:
PSW(0.9689742659805629, [1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1])
Depth 1: [3, 0, 13, 0] points, 16 living leaves...
Depth 2: [13, 15, 13, 13] points, 38 living leaves...
New candidate s.m.p. found because ⟨v_5*, v⟩ - 1 = -2.126040288235251 > 1.0e-6:
aborting with smp candidate PSW(0.9600913508133908, [1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1]) of length 50 after prefixing the suffix with 3 times the current s.m.p. Increase the value of the `max_s

In [26]:
using JuMP, MosekTools
@time smp, done, polys = invariant_polytopes(hs, with_optimizer(Mosek.Optimizer, QUIET=true), psw2,
                                             tol=0.0, verbose=1, max_length=40);

PSW(0.9660273552468552, [3, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1])
PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1])
0 living leaves, PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1]) is a spectral maximizing product (s.m.p.)
Use `tol` < -1.1102230246251565e-16 to eliminate at least one more living leaf.
Use `tol` > 5.133392536249559e-5 to keep at least one more dead leaf.
Use `new_candidate_tol` < -0.0357801298422491 to consider at least one more candidate as s.m.p.
Use `new_candidate_tol` > 0.009191530755863875 to eliminate at least one s.m.p. candidate.
  1.123988 seconds (3.45 M allocations: 258.114 MiB, 6.81% gc time)


## Sum-of-Squares

In [4]:
using MosekTools
using JuMP
factory = with_optimizer(Mosek.Optimizer, QUIET=true);

## Example 3.6

In [9]:
using MultivariatePolynomials
function plot_lyap(p)
    d = mindegree(p)
    @assert d == maxdegree(p)
    @assert nvariables(p) == 2
    α = linspace(0., 2π, 1000)
    X = cos.(α)
    Y = sin.(α)
    λ = map(x -> p(variables(p) => x), zip(X, Y))
    λ = λ.^(1/d)
    plot(X ./ λ, Y ./ λ, aspect_ratio=:equal, label="", grid=false, border=nothing, ticks=nothing, linewidth=3)#, grid=false, label="")
    plot!(X, Y, label="", linestyle=:dash, linecolor=:black, linewidth=2)
end

plot_lyap (generic function with 1 method)

In [ ]:
gr()
for d in 1:6
    for v in 1:4
        plot_lyap(get(s.lyaps[d]).primal[v])
        savefig("PEDJ_d" * string(d) * "_v" * string(v) * ".png")
    end
end

## Example 4.3

In [14]:
using LinearAlgebra
using MultivariatePolynomials
using MultivariateMoments
d = 1
tol = 1e-4
lyap = getlyap(hs, d; factory=factory, tol=tol)
for s in states(hs)
    for t in out_transitions(hs, s)
        σ = symbol(hs, t)
        ν = lyap.dual[t]
        a = extractatoms(ν, tol)
        if a !== nothing
            v = target(hs, t)
            atom = a.atoms[1]
            x = atom.center
            nx = norm(x, 2)
            y = round.(x / nx, digits=3)
            println("μ_{$s$v$σ} = $nx * δ_{$y}")
            @show normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y)
        end
    end
end

μ_{131} = 1.0907033942729127 * δ_{[0.917, 0.399]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.999626, -0.027334]
μ_{312} = 1.143222101937864 * δ_{[0.875, 0.485]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.916808, 0.399328]
μ_{331} = 1.3210537518102323 * δ_{[0.757, -0.653]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.422686, -0.906276]


## Example 4.10

In [57]:
function sequencelb(s, d, l; p_0s = [:Primal, :Random], v_0s = states(s), niter=100)
    best = 0.
    for p_0 in p_0s
        for v_0 in v_0s
            println("d = $d, l = $l, p_0 = $p_0, v_0 = $v_0")
            println("Build sequence")
            @time seq = sosbuildsequence(s, d, p_0=p_0, v_0=v_0, l=l, niter=niter)
            println("Find smp")
            @time psw = findsmp(seq)
            if psw.growthrate ≈ smp.growthrate
                printstyled(psw, bold=true, color=:blue)
                println()
            else
                println(psw)
            end
            @assert psw !== nothing
            best = max(best, psw.growthrate)
        end
    end
    return best
end

sequencelb (generic function with 1 method)

In [58]:
soslb = zeros(6)
sosub = zeros(6)
sosex = zeros(6)
seql = [zeros(6) for i in 1:3];

In [77]:
function find_ub(d; tol=4e-7, reset=true)
    sosdata(hs).lb = 0.0
    if length(sosdata(hs).lyaps) ≥ d && reset
        sosdata(hs).lyaps[d] = nothing
    end
    while length(soslb) < d
        push!(soslb, 0.0)
    end
    while length(sosub) < d
        push!(sosub, 0.0)
    end
    @time soslb[d], sosub[d] = soslyapb(hs, d, factory=factory, tol=tol, ranktols=[], disttols=[], verbose=2)
end

find_ub (generic function with 1 method)

In [83]:
function full_sos(d; ls=1:3, tol=1e-2, reset=true, kwargs...)
    sosdata(hs).smp = nothing
    find_ub(d, tol=tol, reset=reset)
    if hassmp(hs)
        sosex[d] = getsmp(hs).growthrate
    end
    for l in ls
        @time seql[l][d] = sequencelb(hs, d, l; kwargs...)
    end
end

full_sos (generic function with 1 method)

For $d=1$, we find the s.m.p. with $l=3$ even generating a sequence of 9 matrices.

In [84]:
full_sos(1, niter=9)

Lower bound: 0.0
Upper bound: Inf
Log-diff   : Inf > 0.01
Lower bound: 0.0
Upper bound: 1.2264130506891073
Log-diff   : Inf > 0.01
Lower bound: 0.7438571167146476
Upper bound: 1.2264130506891073
Log-diff   : 0.5 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.2264130506891073
Log-diff   : 0.25 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0823057185224616
Log-diff   : 0.125 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0167321299377252
Log-diff   : 0.06249999999999996 > 0.01
Lower bound: 0.985450570895564
Upper bound: 1.0167321299377252
Log-diff   : 0.03124999999999998 > 0.01
Lower bound: 1.0009691593126104
Upper bound: 1.0167321299377252
Log-diff   : 0.015624999999999972 > 0.01
Lower bound: 1.0009691593126104
Upper bound: 1.0088198577297558
Log-diff   : 0.007812500000000047 ≤ 0.01
  0.971582 seconds (417.99 k allocations: 20.456 MiB, 0.87% gc time)
d = 1, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.000380 seconds (5.25 k allocations: 305.453 KiB)
Find smp
  0

For $d = 2$ and $d = 3$, we still find the s.m.p. the same way as with $l = 3$.

In [85]:
full_sos(2)

Lower bound: 0.0
Upper bound: Inf
Log-diff   : Inf > 0.01
Lower bound: 0.0
Upper bound: 1.2264130506891073
Log-diff   : Inf > 0.01
Lower bound: 0.7438571167146476
Upper bound: 1.2264130506891073
Log-diff   : 0.5 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.2264130506891073
Log-diff   : 0.25 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0823057185224616
Log-diff   : 0.125 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0167321299377252
Log-diff   : 0.06249999999999996 > 0.01
Lower bound: 0.985450570895564
Upper bound: 1.0167321299377252
Log-diff   : 0.03124999999999998 > 0.01
Lower bound: 0.985450570895564
Upper bound: 1.0009691593126104
Log-diff   : 0.015625000000000007 > 0.01
Lower bound: 0.985450570895564
Upper bound: 0.9931795555152475
Log-diff   : 0.0078124999999999575 ≤ 0.01
  0.969250 seconds (450.13 k allocations: 23.214 MiB, 0.84% gc time)
d = 2, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.003109 seconds (56.21 k allocations: 3.528 MiB)
Find smp
  0.0

In [86]:
full_sos(3)

Lower bound: 0.0
Upper bound: Inf
Log-diff   : Inf > 0.01
Lower bound: 0.0
Upper bound: 1.2264130506891073
Log-diff   : Inf > 0.01
Lower bound: 0.7438571167146476
Upper bound: 1.2264130506891073
Log-diff   : 0.5 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.2264130506891073
Log-diff   : 0.25 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0823057185224616
Log-diff   : 0.125 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0167321299377252
Log-diff   : 0.06249999999999996 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 0.985450570895564
Log-diff   : 0.031249999999999983 > 0.01
Lower bound: 0.9701725758916283
Upper bound: 0.985450570895564
Log-diff   : 0.01562500000000005 > 0.01
Lower bound: 0.9701725758916283
Upper bound: 0.9777817337113764
Log-diff   : 0.007812500000000066 ≤ 0.01
  0.967412 seconds (501.77 k allocations: 28.118 MiB, 0.66% gc time)
d = 3, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.004653 seconds (80.77 k allocations: 5.186 MiB)
Find smp
  0.0

For $d=4, 5, 6$, we find the s.m.p. with $l=1$ even generating a sequence of 8 matrices.

In [87]:
full_sos(4, niter=8)

Lower bound: 0.0
Upper bound: Inf
Log-diff   : Inf > 0.01
Lower bound: 0.0
Upper bound: 1.2264130506891073
Log-diff   : Inf > 0.01
Lower bound: 0.7438571167146476
Upper bound: 1.2264130506891073
Log-diff   : 0.5 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.2264130506891073
Log-diff   : 0.25 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0823057185224616
Log-diff   : 0.125 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 1.0167321299377252
Log-diff   : 0.06249999999999996 > 0.01
Lower bound: 0.9551314442456673
Upper bound: 0.985450570895564
Log-diff   : 0.031249999999999983 > 0.01
Lower bound: 0.9701725758916283
Upper bound: 0.985450570895564
Log-diff   : 0.01562500000000005 > 0.01
Lower bound: 0.9701725758916283
Upper bound: 0.9777817337113764
Log-diff   : 0.007812500000000066 ≤ 0.01
  1.004379 seconds (560.35 k allocations: 34.662 MiB, 0.61% gc time)
d = 4, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.001003 seconds (15.38 k allocations: 988.969 KiB)
Find smp
  0

In [40]:
full_sos(5, niter=8)

  1.595366 seconds (939.81 k allocations: 64.703 MiB, 1.26% gc time)
d = 5, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.001327 seconds (20.11 k allocations: 1.285 MiB)
Find smp
  0.000061 seconds (515 allocations: 39.016 KiB)
PSW(0.9748171979372074, [1, 2, 3, 1, 1, 1, 1, 2])
d = 5, l = 1, p_0 = Primal, v_0 = 2
Build sequence
  0.001348 seconds (20.11 k allocations: 1.285 MiB)
Find smp
  0.000052 seconds (486 allocations: 36.141 KiB)
PSW(0.9748171979372074, [1, 1, 1, 1, 2, 1, 2, 3])
d = 5, l = 1, p_0 = Primal, v_0 = 3
Build sequence
  0.001300 seconds (20.11 k allocations: 1.285 MiB)
Find smp
  0.000060 seconds (603 allocations: 47.047 KiB)
PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1])
d = 5, l = 1, p_0 = Primal, v_0 = 4
Build sequence
  0.001288 seconds (20.04 k allocations: 1.279 MiB)
Find smp
  0.000047 seconds (450 allocations: 33.234 KiB)
PSW(0.9392550239418472, [1])
d = 5, l = 1, p_0 = Random, v_0 = 1
Build sequence
  0.001264 seconds (20.17 k allocations: 1.291 MiB)
Find 

In [54]:
full_sos(6, niter=8, tol=1e-6)

  2.917041 seconds (1.82 M allocations: 136.722 MiB, 1.71% gc time)
d = 6, l = 1, p_0 = Primal, v_0 = 1
Build sequence
  0.001780 seconds (25.53 k allocations: 1.688 MiB)
Find smp
  0.000065 seconds (515 allocations: 39.016 KiB)
PSW(0.9748171979372074, [1, 2, 3, 1, 1, 1, 1, 2])
d = 6, l = 1, p_0 = Primal, v_0 = 2
Build sequence
  0.001726 seconds (25.53 k allocations: 1.688 MiB)
Find smp
  0.000052 seconds (486 allocations: 36.141 KiB)
PSW(0.9748171979372074, [1, 1, 1, 1, 2, 1, 2, 3])
d = 6, l = 1, p_0 = Primal, v_0 = 3
Build sequence
  0.001647 seconds (25.53 k allocations: 1.688 MiB)
Find smp
  0.000057 seconds (603 allocations: 47.047 KiB)
PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1])
d = 6, l = 1, p_0 = Primal, v_0 = 4
Build sequence
  0.001607 seconds (25.45 k allocations: 1.678 MiB)
Find smp
  0.000044 seconds (450 allocations: 33.234 KiB)
PSW(0.9392550239418472, [1])
d = 6, l = 1, p_0 = Random, v_0 = 1
Build sequence
  0.001571 seconds (25.59 k allocations: 1.694 MiB)
Find s

In [59]:
find_ub(6, tol=1e-6)

  2.937099 seconds (1.82 M allocations: 136.722 MiB, 1.53% gc time)


(0.8997626162054814, 0.9748990169544798)

In [46]:
find_ub(7, tol=4e-7)

  3.594048 seconds (2.35 M allocations: 202.185 MiB, 0.83% gc time)


(0.9100552430286937, 0.9748174360205996)

In [42]:
find_ub(8, tol=5e-7)

  3.768884 seconds (2.63 M allocations: 275.157 MiB, 1.07% gc time)


(0.9179091059554079, 0.9748174360205996)

In [16]:
smp

PSW(0.9748171979372074, [2, 1, 2, 3, 1, 1, 1, 1])

In [55]:
sosub

6-element Array{Float64,1}:
 1.0039059860382262
 0.9864133950211533
 0.9773044177659095
 0.9753974829222168
 0.9753974829222168
 0.9748990169544798

In [105]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [106]:
using FixedPointNumbers
font = Plots.Font("sans-serif",12,:hcenter,:vcenter,0.0,RGB{FixedPointNumbers.Normed{UInt8, 8}}(0.0,0.0,0.0))
plot(1:6, xlabel = "d", sosub, label="SOS UB", legendfont=font, linewidth=3)
plot!(1:6, seql[3], label="Algorithm 1, l=3", linewidth=3)
plot!(1:6, seql[1], label="Algorithm 1, l=1", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")
plot!(1:6, sosex, label="Atom extraction", linewidth=3)
plot!(1:6, soslb, label="SOS LB", linewidth=3)

In [107]:
plot(2:5, xlabel = "d", xticks=2:5, sosub[2:5], label="", linewidth=3)
plot!(2:5, seql[3][2:5], label="", linewidth=3)
plot!(2:5, seql[1][2:5], label="", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")